In [3]:
!pip install category_encoders

     |████████████████████████████████| 82 kB 335 kB/s 


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

#veri seti data frame e yüklenir.
datadf=pd.read_csv("disasters.csv")

#DATA PREPROCESSİNG

#1)Feature engineering
#Başlangıç ve bitiş yılları çok büyük oranla aynı olduğundan o sutunları dusurdum ve ay bilgisini kullanıp mevsim adında yeni bir sutun oluşturdum.
#Bunun için öncelikle NULL olan değerleri doldurdum
datadf.drop(["Start Day","End Day","End Year","Start Year","End Month"],axis=1,inplace=True)
datadf.fillna(value={"Start Month" :datadf["Start Month"].mean().astype(int)},inplace=True)
datadf['Start Month'] = datadf['Start Month'].replace([3,4,5],'Spring')
datadf['Start Month'] = datadf['Start Month'].replace([6,7,8],'Summer')
datadf['Start Month'] = datadf['Start Month'].replace([9,10,11],'Fall')
datadf['Start Month'] = datadf['Start Month'].replace([12,1,2],'Winter')
datadf.rename(columns={"Start Month" :"Season"},inplace=True)

#Country sutunu ile aynı bilgiyi verdiği için Location sutununu dusurduk
datadf.drop(["Location"],axis=1,inplace=True)

#Disaster Magnitude Value sutunu çok büyük oranda boş olduğundan o sutunun birimini gösteren Dis Mag Scale sutunu da dusurulur.
datadf.drop(["Dis Mag Scale"],axis=1,inplace=True)

#Country sutunu ile aynı olduğundan düşürüldü.
datadf.drop(["ISO"],axis=1,inplace=True)

#Continent sutunu ile çok büyük oranda  aynı bilgiyi verdiği için düşürüldü.
datadf.drop(["Region"],axis=1,inplace=True)

#No effected ve Total effected sutunları aynı bilgiyi bize verdiği  için birini düşürdük. Daha buyuk oranda boş olanı dusurduk.
datadf.drop(["No Affected"],axis=1,inplace=True)

# Tüm satırlardaki değeri aynı olduğundan -Natural- Disaster Group sutunu dusurdum.
datadf.drop(["Disaster Group"],axis=1,inplace=True)

#2)Missing data
#doldurulamayacak oranda boş olan columnların veri setinden silinmesi
columnstodrop=["Glide","Disaster Subsubtype","Event Name","Origin","Associated Dis","Associated Dis2","OFDA Response","Appeal","Declaration",
"Aid Contribution","Dis Mag Value","Latitude","Longitude","Local Time","River Basin","No Injured",
"No Homeless","Insured Damages ('000 US$)","Total Damages ('000 US$)","Adm Level","Admin1 Code","Admin2 Code","Geo Locations"]
datadf.drop(columnstodrop,axis=1,inplace=True)   #%51-%96 oranında boş olan columnları veri setinden düşürülür.

#NULL değeri olan columnların doldurulması
#%19 boş olan kategorik sutunun en çok tekrar eden veri ile doldurulması
datadf["Disaster Subtype"].fillna(datadf.groupby('Disaster Subgroup')['Disaster Subtype'].transform(lambda x : next(iter(x.mode()), np.nan)),inplace=True)
datadf=datadf.dropna(subset=["Disaster Subtype"])

#%29 u boş olan nümerik sutunun label ortalamasına göre doldurulması-Max değer ile diğer değerler çok farklı olduğundan normal ortalama kullanmadık-
datadf['Total Deaths'].fillna(datadf.groupby('Disaster Subgroup')['Total Deaths'].transform('mean'),inplace=True)
datadf=datadf.dropna(subset=["Total Deaths"])

#%29 u boş olan nümerik sutunun label ortalamasına göre doldurulması-Max değer ile diğer değerler çok farklı olduğundan normal ortalama kullanmadık-
datadf['Total Affected'].fillna(datadf.groupby('Disaster Subgroup')['Total Affected'].transform('mean'),inplace=True)

#%2 si boş olan nümerik sutunun ortalmaya göre doldurulması
datadf["CPI"].fillna(value={"CPI" :datadf["CPI"].mean()},inplace=True)

#3)Kategorik datanın nümeriğe çevrilmesi
# Label olacak olan sutunu -Disaster Subgroup- Ordinal encoding ile sayılara çevrilmesi
ord_enc = OrdinalEncoder()
datadf["Target Value"] = ord_enc.fit_transform(datadf[["Disaster Subgroup"]])
datadf.drop(["Disaster Subgroup"],axis=1,inplace=True)

#Kategorik datamızda unique değer çok fazla olduğu için direkt olarak One-Hot-Encoding yapılmıyor bu sebeple grupları bir araya topladım.
#Standart %5 ve altının yeni bir grupta toplanması ama ben bizim datamızın özelinde sutunlara göre yeni değerler belirledim.
def combine(percent,name):
    mask = datadf[name].map(datadf[name].value_counts()) < int(datadf.shape[0])*percent
    datadf[name] =  datadf[name].mask(mask, 'other')

combine(0.04,"Disaster Type")
combine(0.04,"Disaster Subtype")
combine(0.02,"Country")

#One Hot Encoding ile kategorik data nümeriğe dönüştürülür
def onehotencode(name):
    encoder=ce.OneHotEncoder(cols=name,handle_unknown='return_nan',return_df=True,use_cat_names=True)
    x = encoder.fit_transform(datadf)
    return x

datadf=onehotencode("Disaster Type")
datadf=onehotencode("Disaster Subtype")
datadf=onehotencode("Country")
datadf=onehotencode("Continent")
datadf=onehotencode("Season")

datadf = datadf.reset_index()

#Modelin oluşturulması
X=datadf[['Year', 'Seq', 'Disaster Type_Drought', 'Disaster Type_Earthquake',
       'Disaster Type_other', 'Disaster Type_Storm', 'Disaster Type_Flood',
       'Disaster Type_Epidemic', 'Disaster Type_Landslide',
       'Disaster Subtype_Drought', 'Disaster Subtype_Ground movement',
       'Disaster Subtype_other', 'Disaster Subtype_Tropical cyclone',
       'Disaster Subtype_Riverine flood', 'Disaster Subtype_Bacterial disease',
       'Disaster Subtype_Convective storm', 'Disaster Subtype_Flash flood',
       'Country_other', 'Country_India', 'Country_Bangladesh', 'Country_China',
       'Country_Indonesia', 'Country_United States of America (the)',
       'Country_Japan', 'Country_Philippines (the)', 'Continent_Africa',
       'Continent_Asia', 'Continent_Americas', 'Continent_Europe',
       'Continent_Oceania', 'Season_Summer', 'Season_Spring', 'Season_Fall',
       'Season_Winter', 'Total Deaths', 'Total Affected', 'CPI']]


y=datadf["Target Value"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80% training and 20% test
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=10)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(y_pred)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


'''corrMatrix = data.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()'''
'''data_crosstab = pd.crosstab(data["Region"], data["Disaster Subgroup"],  margins = False)
print(data_crosstab)'''""""
plt.bar(datadf["Disaster Subgroup"], datadf["Disaster Type"])"""
"""plt.xlabel("Disaster Subgroup", fontsize=8)
plt.ylabel("Disaster Type", fontsize=8)"""
#plt.xticks(index, data_bar.province, fontsize=7, rotation=75)
#4plt.title(‘Number of the confirmed COVID-19 Cases in South Korea’)
""".show()
.boxplot(x = 'day', y = 'total_bill', data = tips)"""

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


FileNotFoundError: ignored